In [1]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel("bert", 'Type_of_the_question/outputs', use_cuda=True)

/home/jovyan/.mlspace/envs/adaptiverag/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from scipy.special import softmax

path = "data/adaptive_rag_musique/train.csv"
cur_data = pd.read_csv(path)

In [3]:
eval_data = []
for i in range(len(cur_data)):
    cur_test = [cur_data.loc[i, 'question_text'], 1]
    eval_data.append(cur_test)

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

In [4]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]/home/jovyan/.mlspace/envs/adaptiverag/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Running Evaluation: 100%|██████████| 5/5 [00:00<00:00,  9.83it/s]


In [5]:
for i in range(len(eval_df)):
    
    eval_df.loc[i, 'prob_generic'] = softmax(model_outputs[i])[0]
    eval_df.loc[i, 'prob_ordinal'] = softmax(model_outputs[i])[1]
    eval_df.loc[i, 'prob_intersection'] = softmax(model_outputs[i])[2]
    eval_df.loc[i, 'prob_superlative'] = softmax(model_outputs[i])[3]
    eval_df.loc[i, 'prob_yesno'] = softmax(model_outputs[i])[4]
    eval_df.loc[i, 'prob_comparative'] = softmax(model_outputs[i])[5]
    eval_df.loc[i, 'prob_multihop'] = softmax(model_outputs[i])[6]
    eval_df.loc[i, 'prob_difference'] = softmax(model_outputs[i])[7]
    eval_df.loc[i, 'prob_count'] = softmax(model_outputs[i])[8]

In [6]:
cur_data['prob_generic'] = eval_df['prob_generic']
cur_data['prob_ordinal'] = eval_df['prob_ordinal']
cur_data['prob_intersection'] = eval_df['prob_intersection']
cur_data['prob_superlative'] = eval_df['prob_superlative']
cur_data['prob_yesno'] = eval_df['prob_yesno']
cur_data['prob_comparative'] = eval_df['prob_comparative']
cur_data['prob_multihop'] = eval_df['prob_multihop']
cur_data['prob_difference'] = eval_df['prob_difference']
cur_data['prob_count'] = eval_df['prob_count']

In [7]:
cur_data.head(n=1)

,qid,question_text,context,answer_objects,reference,rag_one_retrieve,no_retrieve,bela_base,bela_base_ents,bela_base_ents_lbls,...,complexity,prob_generic,prob_ordinal,prob_intersection,prob_superlative,prob_yesno,prob_comparative,prob_multihop,prob_difference,prob_count
0,4hop2__161602_474028_88460_21063,There exists a country A that forms a boundary...,"[{'idx': 0, 'is_supporting': False, 'paragraph...","[{'date': {'day': '', 'month': '', 'year': ''}...",['Yangon Stock Exchange Joint Venture Co. Ltd'],Shanghai Stock Exchange,Lao Securities Exchange,"[{'offsets': [15, 71, 140, 140, 142], 'lengths...","Q47507606, Q5292119, Q568540, Q16365246","2020 AFC U-23 Championship, A Don, country A, A",...,multihop,0.000148,0.000329,0.008584,0.000221,0.000107,0.0001,0.990249,0.000128,0.000135


In [8]:
cur_data.to_csv(path, index = False)